In [1]:
# DOWNLOAD Mistral-DNA GIT REPO
!git clone https://github.com/raphaelmourad/Mistral-DNA.git


Cloning into 'Mistral-DNA'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 331 (delta 27), reused 30 (delta 5), pack-reused 235 (from 1)
Receiving objects: 100% (331/331), 112.69 MiB | 22.64 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [2]:
import sys
!{sys.executable} -m pip install -U transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 34.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 121.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import os

import torch
import numpy as np

from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, AutoConfig
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset

In [2]:
# CHECK GPU
# We can see how many VRAM is used and how much the GPU is used.
!nvidia-smi

Wed Jul 16 03:27:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:81:00.0 Off |                    0 |
| N/A   19C    P8             15W /   70W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Tested with numpy==1.26.4
np.__version__

'1.26.4'

In [4]:
# Tested with transformers==4.47.1
import transformers
transformers.__version__

'4.53.2'

In [2]:
# Tested with accelerate==0.32.1
import accelerate
accelerate.__version__

'1.6.0'

In [3]:
torch.backends.cudnn.benchmark=True
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [4]:
!ls

core.python-7-1752650750-3107-3103-c0605a-s12.ufhpc.1640071
Desktop
do_Llama3_v2.ipynb
embeddings_results_mistral_covid
Mistral-DNA
ondemand
Pat10_72_TM_7188716.log
Pat11_72_TM_7188717.log
Pat13_36_TM_7188734.log
Pat13_72_TM_7188719.log
Pat9_72_TM_7188715.log
tutorial-env
Untitled.ipynb


In [5]:
# SET DIRECTORY
os.chdir("Mistral-DNA/")
print(os.getcwd())

/home/simonerancati/Mistral-DNA


In [6]:
# CHOOSE THE LLM ARCHITECTURE
# To do during class:
# - look at the original archicture of Mixtral-8x7B-v0.1, discuss the model
# - change the model architecture by adding or removing transformer blocks, hidden states, number of attention heads, and number of experts
# - test BERT model architecture?
# NB: flash attention 2 does not work with T4 GPU
config = AutoConfig.from_pretrained("data/models/Mixtral-8x7B-v0.1", cache_dir = "/blue/salemi/share/varcovid") # Mixture of expert
#model = AutoModelForCausalLM.from_config(config,attn_implementation="flash_attention_2")
model = AutoModelForCausalLM.from_config(config,attn_implementation="eager")
model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(4096, 256)
    (layers): ModuleList(
      (0-7): 8 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=256, out_features=256, bias=False)
          (k_proj): Linear(in_features=256, out_features=256, bias=False)
          (v_proj): Linear(in_features=256, out_features=256, bias=False)
          (o_proj): Linear(in_features=256, out_features=256, bias=False)
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=256, out_features=64, bias=False)
          (experts): ModuleList(
            (0-63): 64 x MixtralBlockSparseTop2MLP(
              (w1): Linear(in_features=256, out_features=256, bias=False)
              (w2): Linear(in_features=256, out_features=256, bias=False)
              (w3): Linear(in_features=256, out_features=256, bias=False)
              (act_fn): SiLU()
            )
          )
        )
        (

In [7]:
# LOAD BPE LETTER TOKENIZER
# Question during class: what are UNK, CLS, SEP, PAD and MASK?
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, cache_dir = "/blue/salemi/share/varcovid")
tokenizer.padding_side  = 'left'
print(tokenizer)

tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

PreTrainedTokenizerFast(name_or_path='zhihan1996/DNABERT-2-117M', vocab_size=4096, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [8]:
# DNA ENCODING
# During class change the DNA sequence and see what happens.
# Change max_length, padding, truncation, ...
#encoding = tokenizer("ATTT", padding="longest", truncation=True, return_tensors="pt")
encoding = tokenizer("ATTGTGGGTCCCCGTAGATGATAGGGGCCCCCC", max_length=15, padding='max_length', truncation=True, return_tensors="pt")
print(encoding)

{'input_ids': tensor([[   3,    3,    3,    3,    3,    1, 2061,  281,  485,   35,  245, 2179,
          102,   13,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [9]:
# NUMBER OF MODEL PARAMETERS
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f"Model size: {pytorch_total_params/1000**2:.1f}M parameters")

Model size: 105.0M parameters


In [10]:
# LOAD DATA
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [11]:
dataset_text = load_dataset("csv", data_files="data/genome_sequences/hg38/sequences_hg38_200b_verysmall.csv.gz")
dataset_text['train']['text'][0:5]

Generating train split: 0 examples [00:00, ? examples/s]

['TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCTAACCCTAACCCTAACCCTAA',
 'CCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAAACCCTAAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCAACCCCAACCCCAACCCCAACCCCAACCCCAACCCTAACCCCTAACCCTAACCCTAACCCTACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCC',
 'TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAGCCGGCCCGCCCGCCCGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAGAGTACCACCGAAATCTGTGCAGAGGACAACGCAGCTCCGCCCTCGCGGTGCTCTCCGGGTCTGTGCT',
 'GAGGAGAACGCAACTCCGCCGTTGCAAAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGA',
 'CACATGCTAGCGCGTCGGGGTGGAGGCGTGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGACACATGCTACCGCGTCCAGGGGTGGAGGCGTGGCGCAGGCGCAGAGAGGCGCACCGCGCCGGCGCAGGCGCAGAGACACATGCTAGCGCGTCCAGGGGTGGAGG

In [12]:
# TOKENIZE DATA
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="longest", truncation=True, return_tensors="pt")

dataset = dataset_text.map(tokenize_function, batched=True)
print(dataset["train"])

Map:   0%|          | 0/99999 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 99999
})


In [13]:
train_size = int(0.8 * len(dataset["train"]))
test_size = len(dataset["train"]) - train_size
train_set, val_set = torch.utils.data.random_split(dataset["train"], [train_size, test_size])

In [14]:
# EXAMINE THE DATA
# What is input ids?
# What is attention mask?
train_set[1]

{'text': 'CGCCAGCCTCTGCACCCCTACTGCTGGGCCCCATGGGAGGCGGGCGCCCGGGTGCTCAGAGACCGCGCCCTCCCCGTCAGGCAGACGCTGCTCTGCCTTCCGGTTCCCTTATGGGGCCAGAGTCTATACAAATGAGCACACAGCTACACTGAGGTACACACAACCACAGCCGGAGACCATCCACAGAGCTGGGCCCCACT',
 'input_ids': [3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  1,
  504,
  1411,
  1472,
  236,
  28,
  136,
  249,
  165,
  72,
  601,
  72,
  1994,
  50,
  13,
  601,
  78,
  2796,
  840,
  403,
  178,
  64,
  3160,
  1128,
  97,
  560,
  2666,
  49,
  693,
  44,
  786,
  59,
  138,
  44,
  862,
  231,
  33,
  262,
  401,
  728,
  136,
  34,
  1038,
  2],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [15]:
len(train_set)

79999

In [16]:
train_set[1]["input_ids"]

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 504,
 1411,
 1472,
 236,
 28,
 136,
 249,
 165,
 72,
 601,
 72,
 1994,
 50,
 13,
 601,
 78,
 2796,
 840,
 403,
 178,
 64,
 3160,
 1128,
 97,
 560,
 2666,
 49,
 693,
 44,
 786,
 59,
 138,
 44,
 862,
 231,
 33,
 262,
 401,
 728,
 136,
 34,
 1038,
 2]

In [22]:
# PARAMETERS FOR PRETRAINING
# During class, modify:
# - batchsize
# - number of epochs
# - learning rate
# - weight_decay
# - gradient accumulation steps

batchsize=32
training_args = TrainingArguments(
        output_dir='./results/models',
        eval_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=50,
        per_device_train_batch_size=batchsize,
        per_device_eval_batch_size=batchsize,
        learning_rate=5e-4,
        weight_decay=0.01,
        logging_dir='./logs',
        load_best_model_at_end=True,
        bf16=True,
        gradient_accumulation_steps=50,
        report_to="none",   # <‑‑ disattiva callbacks verso W&B
)
import os
os.environ["WANDB_DISABLED"] = "true"

print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
eval_use_gather_object=False,
fp16=False,
fp16

In [ ]:
# PRETRAIN MODEL
# During class, look at:
# - the use of CPU RAM, GPU RAM
# - estimates how long it takes for 1 epoch

# Don't train until the end as it takes too much time (20h).

# with 105Mb Mixtral model, 110h / 50 epochs (1M sequences) = 2h / epoch

!pip install wandb
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_set,
    eval_dataset=val_set,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

print ('Start a trainer...')
# Start training
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Start a trainer...


Epoch,Training Loss,Validation Loss
1,No log,6.708786
2,No log,6.586504
3,No log,6.451020
4,No log,6.296253
5,No log,6.157484
6,No log,6.061378
7,No log,6.013256
8,No log,5.980645
9,No log,5.994769
